In [3]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features

df_train = pd.read_csv('./data/train.csv')
# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)

# df_train, df_test = features_de_csvs(df_train, df_test)

# df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.15)

In [4]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

## LightGBM model

In [5]:
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):        
        x_train, x_test, y_train, y_test = train_test_split(x, y)
        return super(LightGBMWrapper, self).fit(x_train, y_train)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X, 
               num_iteration=self.best_iteration_)

hps = {'bagging_fraction': 0.8988911725316586,
 'bagging_freq': 22.0,
 'feature_fraction': 0.6622442122619671,
 'learning_rate': 0.16422725363286422,
 'max_depth': 22.0,
 'num_leaves': 180.0,
 'test_size': 0.20892455926004772}

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae', # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': int(hps['num_leaves']),
    'learning_rate': hps['learning_rate'],
    'feature_fraction': hps['feature_fraction'],
    'bagging_fraction': hps['bagging_fraction'],
    'bagging_freq': int(hps['bagging_freq']),
    'max_depth': int(hps['max_depth']),
    'verbose': 0
}

lgb_m = LightGBMWrapper(**params)

## Keras model

In [6]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation

def keras_modelo():    
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'], validation_split=0.1)
    return model

keras_m = KerasRegressor(build_fn=keras_modelo, epochs=10)

Using TensorFlow backend.


## XGBoost

In [7]:
import xgboost as xgb

class XGBoostWrapper(xgb.XGBRegressor):
    
    def fit(self, x, y):
        return super(xgb.XGBRegressor, self).fit(x, y, early_stopping_rounds=2, eval_metric='mae', eval_set=[(x, y)])
    
    def predict(self, X):
        return super(xgb.XGBRegressor, self).predict(X)


hps = {'alpha': 20.91434940058063,
       'colsample_bytree': 0.65,
       'learning_rate': 0.14,
       'max_depth': int(16.0),
       'n_estimators': int(150.0),
       'test_size': 0.2,
       'early_stopping_rounds': 5,
       'n_jobs': 4}


n_estimators = int(hps['n_estimators'])
max_depth = int(hps['max_depth'])

xgb_m = XGBoostWrapper(**hps)

## Stacking

In [8]:
from sklearn.preprocessing import MinMaxScaler

from vecstack import StackingTransformer

features = ['habitaciones', 
            'garages', 
            'banos',
            'antiguedad',
            'metroscubiertos', 
            'metrostotales', 
            'lat', 'lng',
            'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'promedio_precio_ciudad', 
                 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'count_id_zona', 'count_ciudad', 'puntaje', 
                     'count_tipo_propiedad_ciudad', 
#                  'varianza_id_zona',
#                  'metros_cubiertos_normalizados',
                 'promedio_precio_tipo_propiedad_ciudad_gen',
#                  'promedio_precio_hbg_tipo_propiedad_provincia',
#                  'diferencia_metros',
                 'count_id_zona'
                 'dias_desde_datos',
                 'meses_desde_datos',
                 'porcentaje_metros',
#                  'promedio_por_mes', 
#                  'count_tipo_propiedad',
#                  'tam_ambientes',
                 'promedio_precio_hbg_tipo_propiedad']


features += features_test

features += cols_tipodepropiedad_ohe + cols_provincia_ohe + cols_zona_ohe


x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

modelos = [('xgboost', xgb_m), 
#            ('keras', keras_m), 
           ('lightgbm', lgb_m)]

stack = StackingTransformer(modelos, regression=True, verbose=2, n_folds=6)

stack = stack.fit(x_train, y_train)

s_train = stack.transform(x_train)
s_test = stack.transform(x_test)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [xgboost: XGBoostWrapper]
[21:31:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:2.18966e+06
Will train until validation_0-mae hasn't improved in 2 rounds.
[1]	validation_0-mae:1.8909e+06
[2]	validation_0-mae:1.63406e+06
[3]	validation_0-mae:1.41311e+06
[4]	validation_0-mae:1.22468e+06
[5]	validation_0-mae:1.06398e+06
[6]	validation_0-mae:926426
[7]	validation_0-mae:809224
[8]	validation_0-mae:710350
[9]	validation_0-mae:626254
[10]	validation_0-mae:556120
[11]	validation_0-mae:496360
[12]	validation_0-mae:446509
[13]	validation_0-mae:404355
[14]	validation_0-mae:368831
[15]	validation_0-mae:339917
[16]	validation_0-mae:316522
[17]	validation_0-mae:295993
[18]	validation_0-mae:278357
[19]	validation_0-mae:264246
[20]	validation_0-mae:251036
[21]	validation_0-mae:239954
[22]	validati

[108]	validation_0-mae:88798.6
[109]	validation_0-mae:87993.7
[110]	validation_0-mae:87785.9
[111]	validation_0-mae:87261.1
[112]	validation_0-mae:87061.5
[113]	validation_0-mae:86876.5
[114]	validation_0-mae:86348.4
[115]	validation_0-mae:86263.1
[116]	validation_0-mae:85733.1
[117]	validation_0-mae:85409.7
[118]	validation_0-mae:85295.7
[119]	validation_0-mae:84835.9
[120]	validation_0-mae:84798.4
[121]	validation_0-mae:84380.6
[122]	validation_0-mae:84208.8
[123]	validation_0-mae:84046
[124]	validation_0-mae:83396.6
[125]	validation_0-mae:83281.5
[126]	validation_0-mae:82988.5
[127]	validation_0-mae:82539.9
[128]	validation_0-mae:82304.7
[129]	validation_0-mae:81822.2
[130]	validation_0-mae:81242.3
[131]	validation_0-mae:80769.5
[132]	validation_0-mae:80213.8
[133]	validation_0-mae:80007.3
[134]	validation_0-mae:79249.1
[135]	validation_0-mae:78929
[136]	validation_0-mae:78696.9
[137]	validation_0-mae:78500.2
[138]	validation_0-mae:78272.2
[139]	validation_0-mae:77774.4
[140]	valida

[68]	validation_0-mae:110151
[69]	validation_0-mae:109533
[70]	validation_0-mae:108322
[71]	validation_0-mae:107686
[72]	validation_0-mae:107392
[73]	validation_0-mae:107086
[74]	validation_0-mae:106588
[75]	validation_0-mae:105987
[76]	validation_0-mae:105343
[77]	validation_0-mae:104533
[78]	validation_0-mae:103999
[79]	validation_0-mae:103565
[80]	validation_0-mae:103295
[81]	validation_0-mae:102511
[82]	validation_0-mae:102121
[83]	validation_0-mae:101370
[84]	validation_0-mae:101046
[85]	validation_0-mae:100952
[86]	validation_0-mae:100568
[87]	validation_0-mae:99048.7
[88]	validation_0-mae:98370.1
[89]	validation_0-mae:97906.2
[90]	validation_0-mae:97247.6
[91]	validation_0-mae:96798.5
[92]	validation_0-mae:96004.1
[93]	validation_0-mae:95298.8
[94]	validation_0-mae:95173.4
[95]	validation_0-mae:95126.8
[96]	validation_0-mae:94700.1
[97]	validation_0-mae:94287.3
[98]	validation_0-mae:93039.5
[99]	validation_0-mae:92667
[100]	validation_0-mae:92090.3
[101]	validation_0-mae:91511.4

[27]	validation_0-mae:195512
[28]	validation_0-mae:190450
[29]	validation_0-mae:185358
[30]	validation_0-mae:180873
[31]	validation_0-mae:176480
[32]	validation_0-mae:173470
[33]	validation_0-mae:169726
[34]	validation_0-mae:167186
[35]	validation_0-mae:163983
[36]	validation_0-mae:161109
[37]	validation_0-mae:159282
[38]	validation_0-mae:156276
[39]	validation_0-mae:154029
[40]	validation_0-mae:152081
[41]	validation_0-mae:150099
[42]	validation_0-mae:147924
[43]	validation_0-mae:144798
[44]	validation_0-mae:142347
[45]	validation_0-mae:140326
[46]	validation_0-mae:138693
[47]	validation_0-mae:136931
[48]	validation_0-mae:135465
[49]	validation_0-mae:133052
[50]	validation_0-mae:130851
[51]	validation_0-mae:129182
[52]	validation_0-mae:127745
[53]	validation_0-mae:126494
[54]	validation_0-mae:124333
[55]	validation_0-mae:122995
[56]	validation_0-mae:121796
[57]	validation_0-mae:120463
[58]	validation_0-mae:119701
[59]	validation_0-mae:117974
[60]	validation_0-mae:116820
[61]	validatio

In [9]:
s_train = stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features))
s_test = stack.transform(utils.filtrar_features(df_test_f, features))

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



## Prediccion con todos los features + stacking

In [10]:
df_train_s = df_train_f.copy()
df_test_s = df_test_f.copy()

df_train_s['stack01'], df_train_s['stack02'] = zip(*s_train)
df_test_s['stack01'], df_test_s['stack02'] = zip(*s_test)

In [11]:
df_train_s['id'] = df_train['id']
df_test_s['id'] = df_test['id']

In [12]:
params_2nd = {'bagging_fraction': 0.7740520226030885,
 'bagging_freq': int(7.0),
 'feature_fraction': 0.8422472893793045,
 'learning_rate': 0.1508386725397851,
 'max_depth': int(12.0),
 'num_leaves': int(110.0)}

lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(utils.filtrar_features(df_train_s, features + ['stack01', 'stack02']), df_train['precio'].values)

LightGBMWrapper(bagging_fraction=0.7740520226030885, bagging_freq=7,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.8422472893793045, importance_type='split',
                learning_rate=0.1508386725397851, max_depth=12,
                min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=110,
                objective=None, random_state=None, reg_alpha=0.0,
                reg_lambda=0.0, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0)

In [14]:
df_test_s['target'] = lgb_m_2nd.predict(utils.filtrar_features(df_test_s, features + ['stack01', 'stack02']))
df_test_s[['id', 'target']].to_csv('respuesta24.csv', index = False)

## Prediccion solo con features de stacking

In [17]:
params_2nd = {'bagging_fraction': 0.8924398062087346,
 'bagging_freq': int(36.0),
 'feature_fraction': 0.16167385124183287,
 'learning_rate': 0.054693418899570134,
 'max_depth': int(4.0),
 'num_leaves': int(93.0)}

lgb_m_2nd = LightGBMWrapper(**params_2nd)
lgb_m_2nd.fit(stack.transform(utils.filtrar_features(df_train_f.drop('precio', axis=1), features)), df_train_f['precio'].values)

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



LightGBMWrapper(bagging_fraction=0.8924398062087346, bagging_freq=36,
                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                feature_fraction=0.16167385124183287, importance_type='split',
                learning_rate=0.054693418899570134, max_depth=4,
                min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=93,
                objective=None, random_state=None, reg_alpha=0.0,
                reg_lambda=0.0, silent=True, subsample=1.0,
                subsample_for_bin=200000, subsample_freq=0)

In [24]:
keras_mae_train = utils.MAE(y_train, lgb_m_2nd.predict(stack.transform(x_train)))
keras_mae_test = utils.MAE(y_test, lgb_m_2nd.predict(stack.transform(x_test)))
print(f"MAE Stacking (train): {keras_mae_train:.5f}")
print(f"MAE Stacking (test): {keras_mae_test:.5f}")

Train set was detected.
Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

MAE St

In [26]:
s_test_f = stack.transform(utils.filtrar_features(df_test_f, features))
y_pred_test_f = lgb_m_2nd.predict(s_test_f)
df_test_f['target'] = y_pred_test_f
df_test_f[['id', 'target']].to_csv('respuesta21.csv', index = False)

Transforming...

estimator  0: [xgboost: XGBoostWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE

estimator  1: [lightgbm: LightGBMWrapper]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    model from fold  5: done
    ----
    DONE



In [13]:
features = ['stack01', 'stack02', 'stack03']

def eval_lightgbm(args):
    num_leaves, learning_rate, feature_fraction, bagging_fraction, bagging_freq, max_depth = args

    lgb_train = lgb.Dataset(s_train, y_train)
#     lgb_eval = lgb.Dataset(s_test, y_test, reference=lgb_train)
    
    num_leaves = int(num_leaves)
    bagging_freq = int(bagging_freq)
    max_depth = int(max_depth)
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': bagging_freq,
        'max_depth': max_depth,
        'verbose': -1,
    }

    gbm = lgb.train(params,
                    lgb_train,
#                     valid_sets=lgb_eval,
                    num_boost_round=250,
#                     early_stopping_rounds=15,
                    verbose_eval=-1)
    
    y_pred_test = gbm.predict(s_test, num_iteration=gbm.best_iteration)
    return utils.MAE(y_test, y_pred_test)

space = [hp.quniform('num_leaves', 30, 130, 1), hp.uniform('learning_rate', 0.05, 0.9),
        hp.uniform('feature_fraction', 0.10, 0.90), hp.uniform('bagging_fraction', 0.10, 0.90),
        hp.quniform('bagging_freq', 1, 130, 1), hp.quniform('max_depth', 1, 20, 1)]

hps = fmin(eval_lightgbm, space=space, algo=tpe.suggest, max_evals=100, verbose=1)

display(hps)

 12%|█▏        | 12/100 [00:48<05:53,  4.02s/it, best loss: 437714.00137358136]


KeyboardInterrupt: 

In [141]:

# keras_mae_train = utils.MAE(y_test, lgb_m.predict(x_test_s))
# print(f"MAE Keras (train): {keras_mae_train:.5f}")

MAE Keras (train): 524925.45271
